In [17]:
import json
import librosa
import requests
import random
import re
import os
import IPython.display as ipd
from pydub import AudioSegment

/home/victor/.local/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [14]:
dataset_base_path = "./audio_files_raw/Iteration_02/audio_dataset"
files = os.listdir(dataset_base_path)

print(len(files))

20


In [8]:
with open("raw_dataset.txt","w") as f:
    f.write(" | \n".join(files))

In [15]:
with open("./audio_files_raw/Iteration_02/transcripts.txt","r") as f:
    data = f.readlines()

print(len(data))

20


In [9]:
data[0]

'01.wav | The body feels under threat even though there is no threat. This is natural and can happen to anyone. Before meditating, my anxiety was high. I used to worry about my future a lot.\n'

In [18]:
output_path = "./audio_files_raw/Iteration_02/audio_dataset_mono"
for f in data :
    i = f.split('|')
    i = i[0].strip()
    sound = AudioSegment.from_wav(os.path.join(dataset_base_path,i))
    sound = sound.set_channels(1)
    sound.export(os.path.join(output_path,i), format="wav")

In [19]:
files_details = []

for i in data:
    filename = i.split('|')
    audio_filepath = os.path.join(output_path, filename[0].strip())
    duration = librosa.get_duration(filename=audio_filepath)
    text = filename[1].strip('\n')

    r = {
            "audio_filepath" : audio_filepath,
            "text" : text,
            "duration" : round(duration,1),
            "text_no_preprocessing" : text,
            "text_no_preprocessing" : re.sub(r"[^a-z]"," ",text).rstrip(" ")+",",
            "text_normalized" : re.sub(r"[^a-z]"," ",text).rstrip(" ")+","
        }

    files_details.append(r)

print(len(files_details))

20


In [20]:
percentage_training_data = 0.95

In [12]:
random.shuffle(files_details)

# get train, test, validation splits
num_clips = len(files_details)
train_end = int(num_clips * percentage_training_data)

train = files_details[:train_end]
validation = files_details[train_end:]

In [13]:
train_manifest = './fastpitch_train.json'
with open(train_manifest, "w") as f:
    for r in train:
        f.write(json.dumps(r) + '\n')

val_manifest = './fastpitch_val.json'
with open(val_manifest, "w") as f:
    for r in validation:
        f.write(json.dumps(r) + '\n')